# Higher Order Networks of User Consuption of Songs

**Imports**

In [7]:
# Statistical and data framing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Deals with Path related stuff
from pathlib import Path

# Higher order Networks
import pathpy as pp

**IPython Magic Functions**

In [8]:
# will make plot outputs appear and be stored within the notebook.
%matplotlib inline 

# Defaults fot better plots
plt.rcParams['figure.figsize']  = (18, 10)
plt.rcParams['axes.labelsize']  = 20
plt.rcParams['axes.titlesize']  = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['lines.linewidth'] = 4

# Turn on interactive mode
plt.ion() 
plt.style.use('seaborn-colorblind')
plt.rcParams['figure.figsize']  = (12, 8)

In [11]:
dataset_name = "music4all"
dataset_relative_location = "../../dataset"
dataset_path = Path(dataset_relative_location) / dataset_name

### Get dataset CSVs path
csvsPattern = "*.csv"
csvs = list(dataset_path.glob(csvsPattern))
csvs

[PosixPath('../../dataset/music4all/id_tags.csv'),
 PosixPath('../../dataset/music4all/id_metadata.csv'),
 PosixPath('../../dataset/music4all/id_information.csv'),
 PosixPath('../../dataset/music4all/listening_history.csv'),
 PosixPath('../../dataset/music4all/id_lang.csv'),
 PosixPath('../../dataset/music4all/id_genres.csv')]

In [12]:
# Load datasets
datasets = {}

for csv in csvs:
    datasets[csv.stem] = pd.read_csv(csv, delimiter="\t")

In [13]:
# Generate song information, name, artist, album, genre
song_info = pd.merge(datasets['id_information'], datasets['id_genres'], on='id')

# load listening history of the users
lh = datasets['listening_history']


In [14]:
# User history into HON

# Get first user 
user1 = lh[lh["user"] == "user_007XIjOr"]

# Rename column to merge df
renamed_df = user1.rename(columns = {'song':'id'})
user1_songs = pd.merge(renamed_df, song_info, on='id')

In [15]:
def get_transition_timestamps(df):
    list_of_lists = []
    
    # first row is a transition to itself, # source, # target, # timestamp
    list_of_lists.append([df.loc[0, "song"], df.loc[0, "song"],  df.loc[0, "timestamp"]])

    # from 0 to n-1
    for i in range(0, len(df) - 1) :
        list_of_lists.append([df.loc[i, "song"], df.loc[i + 1, "song"],  df.loc[i + 1, "timestamp"]])

    return list_of_lists    

def get_transition_df(dataframe):
    return pd.DataFrame(get_transition_timestamps(dataframe), columns=["source", "target", "timestamp"])

In [20]:
transition_df = get_transition_df(user1_songs)

In [176]:
temporal_net = pp.TemporalNetwork()
# fill the temporal net
count = 0
for row in transition_df.itertuples(index = True, name ='Pandas'):
    print(getattr(row, "source"), getattr(row, "target"), getattr(row, "timestamp"))
    #temporal_net.add_edge(getattr(row, "source")[:5], getattr(row, "target")[:5], count)#getattr(row, "timestamp"), timestamp_format='%Y-%m-%d %H:%M')
    temporal_net.add_edge(getattr(row, "source")[:4], getattr(row, "target")[:4], getattr(row, "timestamp"), timestamp_format='%Y-%m-%d %H:%M')
    count += 1

Your Best American Girl Your Best American Girl 2019-02-20 12:28
Your Best American Girl Your Best American Girl 2019-02-26 18:09
Your Best American Girl LIKE HELL 2019-02-20 12:35
LIKE HELL LIKE HELL 2019-02-25 13:25
LIKE HELL LIKE HELL 2019-02-28 23:25
LIKE HELL LIKE HELL 2019-02-28 23:27
LIKE HELL LIKE HELL 2019-02-28 23:28
LIKE HELL LIKE HELL 2019-03-02 00:13
LIKE HELL LIKE HELL 2019-03-05 12:54
LIKE HELL The Saxophone Song 2019-02-20 12:48
The Saxophone Song Gurdjieff's Daughter 2019-02-20 12:52
Gurdjieff's Daughter Gurdjieff's Daughter 2019-02-21 14:35
Gurdjieff's Daughter The Wedding List 2019-02-20 13:09
The Wedding List The Wedding List 2019-02-20 13:13
The Wedding List The Wedding List 2019-02-22 12:59
The Wedding List The Wedding List 2019-02-25 02:18
The Wedding List Les Jeux to You 2019-02-20 13:33
Les Jeux to You Les Jeux to You 2019-02-20 13:39
Les Jeux to You Les Jeux to You 2019-02-20 13:45
Les Jeux to You Les Jeux to You 2019-02-20 13:51
Les Jeux to You Les Jeux to Yo

Unravel Unravel 2019-03-04 13:57
Unravel Unravel 2019-03-04 13:57
Unravel Just You And Me 2019-02-28 21:51
Just You And Me Just You And Me 2019-02-28 23:13
Just You And Me Just You And Me 2019-03-05 00:12
Just You And Me John Wayne Gacy, Jr. 2019-02-28 23:30
John Wayne Gacy, Jr. John Wayne Gacy, Jr. 2019-02-28 23:32
John Wayne Gacy, Jr. John Wayne Gacy, Jr. 2019-03-04 12:50
John Wayne Gacy, Jr. John Wayne Gacy, Jr. 2019-03-04 12:50
John Wayne Gacy, Jr. Mykonos 2019-02-28 23:39
Mykonos Mykonos 2019-02-28 23:40
Mykonos Gloria: In Excelsis Deo 2019-02-28 23:49
Gloria: In Excelsis Deo Gloria: In Excelsis Deo 2019-02-28 23:50
Gloria: In Excelsis Deo Gloria: In Excelsis Deo 2019-03-04 16:44
Gloria: In Excelsis Deo Gloria: In Excelsis Deo 2019-03-04 16:44
Gloria: In Excelsis Deo Too Much 2019-03-01 00:24
Too Much Too Much 2019-03-01 00:30
Too Much White Winter Hymnal 2019-03-01 00:27
White Winter Hymnal White Winter Hymnal 2019-03-01 00:37
White Winter Hymnal White Winter Hymnal 2019-03-02 01

In [177]:
print(temporal_net)

Nodes:			142
Time-stamped links:	440
Links/Nodes:		3.0985915492957745
Observation period:	[1550676480, 1551843900]
Observation length:	 1167420 
Time stamps:		 365 
Avg. inter-event dt:	 3207.197802197802
Min/Max inter-event dt:	 60/78660


In [178]:
for e in temporal_net.tedges:
    print(e)

('Your', 'Your', 1550676480)
('Your', 'Your', 1551215340)
('Your', 'LIKE', 1550676900)
('LIKE', 'LIKE', 1551111900)
('LIKE', 'LIKE', 1551407100)
('LIKE', 'LIKE', 1551407220)
('LIKE', 'LIKE', 1551407280)
('LIKE', 'LIKE', 1551496380)
('LIKE', 'LIKE', 1551801240)
('LIKE', 'The ', 1550677680)
('The ', 'Gurd', 1550677920)
('Gurd', 'Gurd', 1550770500)
('Gurd', 'The ', 1550678940)
('The ', 'The ', 1550679180)
('The ', 'The ', 1550851140)
('The ', 'The ', 1551071880)
('The ', 'Les ', 1550680380)
('Les ', 'Les ', 1550680740)
('Les ', 'Les ', 1550681100)
('Les ', 'Les ', 1550681460)
('Les ', 'Les ', 1550681880)
('Les ', 'Les ', 1550767740)
('Les ', 'Les ', 1550854140)
('Les ', 'Les ', 1550854920)
('Les ', 'Les ', 1550857740)
('Les ', 'Les ', 1550858100)
('Les ', 'Les ', 1550892240)
('Les ', 'Les ', 1550892660)
('Les ', 'Les ', 1550971080)
('Les ', 'Les ', 1550980260)
('Les ', 'Les ', 1550980620)
('Les ', 'Les ', 1551048600)
('Les ', 'Les ', 1551048960)
('Les ', 'Les ', 1551049380)
('Les ', 'Les 

In [ ]:
paths = pp.path_extraction.paths_from_temporal_network_dag(temporal_net, delta=3600)

In [183]:
help(pp.path_extraction.paths_from_temporal_network_dag)

TypeError: __call__() got an unexpected keyword argument 'delta'

In [ ]:
mog = pp.MultiOrderModel(paths, 3)

In [ ]:
mog.estimate_order()